In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys, os,git
from pathlib import Path
try:
    import yaml
    import f90nml  # from https://f90nml.readthedocs.io/en/latest/
except ImportError:  # BUG: don't get this exception if payu module loaded, even if on python 2.6.6
    print('\nFatal error: modules not available.')
    print('On NCI, do the following and try again:')
    print('   module use /g/data/hh5/public/modules; module load conda/analysis3\n')
    raise

In [21]:
# ======================================================
# from https://gist.github.com/paulkernstock/6df1c7ad37fd71b1da3cb05e70b9f522
from yaml.representer import SafeRepresenter


class LiteralString(str):
    pass


def change_style(style, representer):
    def new_representer(dumper, data):
        scalar = representer(dumper, data)
        scalar.style = style
        return scalar
    return new_representer

represent_literal_str = change_style('|', SafeRepresenter.represent_str)
yaml.add_representer(LiteralString, represent_literal_str)
# ======================================================

In [22]:
dir_ensemble = '/g/data/tm70/ml0072/COMMON/ensemble'
sys.path.append(dir_ensemble)

dir_utils = '/g/data/tm70/ml0072/COMMON/om3-utils/om3utils'
sys.path.append(dir_utils)

In [23]:
yaml_text = 'ensemble.yaml'
yamlfile = Path(dir_ensemble,yaml_text)
indata = yaml.load(open(yamlfile, 'r'), Loader=yaml.SafeLoader)
indata

{'template': 'test/1deg_jra55_iaf',
 'startfrom': 'rest',
 'nruns': 0,
 'namelists': {'ice/cice_in.nml': {'shortwave_nml': {'albicei': [0.3,
     0.44,
     0.5],
    'albicev': []},
   'dynamics_nml': {'turning_angle': [-90, 0, 90], 'dragio': []}},
  'ocean/input.nml': {'auscom_ice_nml': {'aice_cutoff': []}}}}

In [28]:
template = indata['template']
print(f'template: {template}')
templatepath = Path(dir_ensemble,template)
print(f'templatepath: {templatepath}')
# templaterepo = git.Repo(templatepath)
# print(templaterepo)
# print(f'current branch: {templaterepo.active_branch.name}')

template: test/1deg_jra55_iaf
templatepath: /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_iaf
